<a href="https://colab.research.google.com/github/adindasyv/2141720096-machinelearning-2023/blob/master/Week%2010/praktikum2_tugas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Praktikum 2 : Generator Teks dengan RNN

### Setup
Import tensorflow

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time

Download Dataset Shakespeare

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 1s 1us/step


Load Data

In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


### Olah Teks

Vectorize Teks <br>
Sebelum training, Anda perlu mengonversi string menjadi representasi numerik. tf.keras.layers.StringLookup dapat mengubah setiap karakter menjadi ID numerik. Caranya adalah teks akan dipecah menjadi token terlebih dahulu.

In [ ]:
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

sekarang buat tf.keras.layers.StringLookup layer

In [ ]:
ids_from_chars = tf.keras.layers.StringLookup(
vocabulary=list(vocab), mask_token=None)

perintah diatas mengconvert token menjadi id

In [ ]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

Karena tujuan tutorial ini adalah untuk menghasilkan teks, penting juga untuk membalikkan representasi ini. Untuk ini Anda dapat menggunakan kode

In [ ]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

Lapisan ini mengconvert kembali karakter dari vektor ID, dan mengembalikannya sebagai karakter tf.RaggedTensor

In [ ]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Anda dapat menggunakan tf.strings.reduce_join untuk menggabungkan kembali karakter menjadi string.

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### Prediksi
Membuat Training Set dan Target


In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [ ]:
seq_length = 100

Metode batch memungkinkan Anda dengan mudah mengonversi karakter individual ini menjadi urutan ukuran yang diinginkan.

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


akan lebih mudah untuk melihat apa yang dilakukan jika Anda menggabungkan token kembali menjadi string

In [ ]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


Untuk pelatihan, Anda memerlukan kumpulan data pasangan (input, label). Dimana input dan label merupakan urutan. Pada setiap langkah waktu, inputnya adalah karakter saat ini dan labelnya adalah karakter berikutnya. Berikut adalah fungsi yang mengambil urutan sebagai masukan, menduplikasi, dan menggesernya untuk menyelaraskan masukan dan label untuk setiap langkah waktu

In [ ]:
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text

In [ ]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
  print("Input :", text_from_ids(input_example).numpy())
  print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


Membuat Batch Training

Anda menggunakan tf.data untuk membagi teks menjadi sequence yang dapat diatur. Namun sebelum memasukkan data ini ke dalam model, Anda perlu mengacak data dan mengemasnya ke dalam batch.

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

### Buat Model

In [ ]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

### Uji Model

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Untuk mendapatkan prediksi aktual dari model, Anda perlu mengambil sampel dari distribusi keluaran, untuk mendapatkan indeks karakter aktual. Distribusi ini ditentukan oleh logit pada kosakata karakter. Catatan: Penting untuk mengambil sampel dari distribusi ini karena mengambil argmax dari distribusi tersebut dapat dengan mudah membuat model terjebak dalam infinote loop. Cobalah untuk contoh pertama di batch

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0],num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

Hal ini memberi kita, pada setiap langkah waktu, prediksi indeks karakter berikutnya:

In [ ]:
sampled_indices

array([23, 30, 24, 54, 55, 13,  1, 53, 65,  8, 32, 32, 31, 19, 14, 32, 33,
       55, 28,  7, 14, 38, 54, 56, 38, 51, 58,  3, 34, 46, 29, 10, 26, 10,
       29, 62, 24, 16, 32, 52, 12, 16, 15, 35, 25,  9, 38, 63, 65, 50, 14,
       16, 48, 30,  8, 57,  1, 56, 12, 25, 44, 53, 31,  5, 39, 47, 52, 28,
       45, 42, 65,  3, 43, 54, 37, 36, 15, 40, 23, 45,  6, 63, 39, 33, 25,
        9,  9, 41, 46, 26, 20, 29, 18, 16,  7, 27, 38, 47,  6, 20])

Dekode kode berikut untuk melihat teks yang diprediksi oleh model tidak terlatih ini:

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'ell;\nUnless you please to enter in the castle\nAnd there repose you for this night.\n\nHENRY BOLINGBROK'

Next Char Predictions:
 b"JQKop?\nnz-SSRFASTpO,AYoqYls!UgP3M3PwKCSm;CBVL.YxzkACiQ-r\nq;LenR&ZhmOfcz!doXWBaJf'xZTL..bgMGPEC,NYh'G"


### Train Model
Tambahan optimizer dan fungsi loss

loss function tf.keras.losses.sparse_categorical_crossentropy standar berfungsi dalam kasus ini karena diterapkan di seluruh dimensi terakhir prediksi. Karena model Anda mengembalikan logits, Anda perlu mengatur flag from_logits.

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1897326, shape=(), dtype=float32)


Model yang baru diinisialisasi tidak boleh terlalu yakin dengan dirinya sendiri, semua log keluaran harus memiliki besaran yang sama. Untuk mengonfirmasi hal ini, Anda dapat memeriksa bahwa eksponensial dari loss rata-rata harus kira-kira sama dengan ukuran kosakata. Loss yang jauh lebih tinggi berarti model tersebut yakin akan jawaban yang salah, dan memiliki inisialisasi yang buruk:

In [ ]:
tf.exp(example_batch_mean_loss).numpy()

66.005135

Konfigurasikan prosedur pelatihan menggunakan metode tf.keras.Model.compile. Gunakan tf.keras.optimizers.Adam dengan argumen default dan fungsi loss.

In [ ]:
model.compile(optimizer='adam', loss=loss)

Konfigurasi Checkpoints

Gunakan tf.keras.callbacks.ModelCheckpoint untuk memastikan bahwa checkpoint disimpan selama pelatihan:

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

Lakukan Proses Training

In [ ]:
EPOCHS = 10

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 16s 58ms/step - loss: 2.7369
Epoch 2/10
172/172 [==============================] - 12s 54ms/step - loss: 1.9925
Epoch 3/10
172/172 [==============================] - 11s 53ms/step - loss: 1.7114
Epoch 4/10
172/172 [==============================] - 11s 54ms/step - loss: 1.5515
Epoch 5/10
172/172 [==============================] - 11s 55ms/step - loss: 1.4521
Epoch 6/10
172/172 [==============================] - 11s 55ms/step - loss: 1.3842
Epoch 7/10
172/172 [==============================] - 11s 56ms/step - loss: 1.3314
Epoch 8/10
172/172 [==============================] - 11s 56ms/step - loss: 1.2867
Epoch 9/10
172/172 [==============================] - 11s 57ms/step - loss: 1.2470
Epoch 10/10
172/172 [==============================] - 11s 56ms/step - loss: 1.2065


### Generate Teks

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:.X,R
OAnYJQRRFUAPrFQbR:A;UkKVj,Oq
bTBPdUGwT!bQhfgiWaB bNj'uAOVs!!d,o-NbmXrLePMmRWXZF,J.JPGyZ&DWOZMO.XDpiJtmuYFk,veDBH&F:da?ATAcgcMktZxivbk!NqoqXZOMTM?aSn3';PCd$q jVpxPN
ahB R.3UnMnlaXZv;SoNjx? Pqk?vriYA?AlcKZdcscIWPz:y
I$,vWgM$;lffxPraVW'VpaMSS;AJ&bgH,NWMkUohH?mK$X:vfPkBv'mBV NMfuvRfyXv$Vr.tvlJsS:B
FUYzVYCmi!HkpbJ$JzAs qZ;ndYMfDp,XIJtsAaITiXu3qoIWVdLHyfACQJt',qRoz3,&yj&.REnZHEi O!RQ!sI!meI?y3v;b&ExWRY' gJJt.SNvqitwvBfQbw$iQwhA3CSlnMzlLHEEiL; q!E-z:CX?Sd!mjmTxvlVSDTcTQl&HT'.KWdfOlkzcHm yVx
G-DF!W:
r-DvKoXmXadMQqEsF hZf3q;iTgTU&rXRnSlVpmlp;drmlUFOqgOLSN, Q?!emvP&RrlJ-q,EBy,q,VTV?'LvZgH
je$:oV?aMAoMjR.'jqEoJpDln
iiTlX VjHDx$ro'IjQ lNR:nEqONh-LlBu:sxxsubfFvDj.FPmdk;!ouGij-PaBmvDCsc'$bZHnxssddMO,&hC3fW Kk;WewRBhkQTR? $WTckcFnznVXshSrYEk&gjoKNOCM,SaCLO&qPm?W,Y OLPhRmrMRtuvzTYSVcmetOgsL,;vZOmPMi?NP$w$maCjtmXD
hWwSJIv
iAFC vrVCDMe!jvgwVU;IvHUwzdFflrOLeHZWqkWolaU&GqX$X.Yd&-yHRl,!
pyK?Sq
pFc&!Uq'$Er?P;CuFNkX$rzmCCze
Kxr AmoMEa$b!.ULzv!ZVnEB3XmAGegmEv,TD3OvNceXlXV,O!!YVbo?Oaya$?,jjX!tvA&OL-

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:,fMlM\nsXqltBZC'yayuojhF&-,.dcBvsbTgaLTRgQfrzokkUpyH-'Hb!dc.BH$p,qABZMJQ3qclCs-ZY!Bz-Ovu?N- Npp$:dpyKrP'-vOxdF\nd'rnzhYWGfywCIGUg3AezZgxh!h\nm!xYW.DUE,GNBvo-I ZNdaMC?;lQ?'J&A&'.EwnYt-acwNCCzPWSJI\nnHY'W:il \ny,RRI?QFrOl&aLMKYKVVO\nSf3Fkzbkx?$XrWIMVm?,IPJeuea;Q-zIj!$!uB;w& mguxoLnpO!RWc''pYXixBrL-LUgBiEfr,s:NRRk-aANHiy:kA$GxXkJCJkR-fBb$yyiEm zVhB::F!UhIw\n$XCB. SIXFL!,AZHiVtAfHIdUsqCzWtM:MUEkejbQhhXdzp!ykZqRo&dQnsvFbGiqZ.VtmabaBz-AwKq,uivIhZlj?f.udKo!\nXPJIC;Gz:OZALVLKQpqGlbSP pv;reDfbG!hVq.'tSX;r3qDiqlGMlyPPv WcrGZX j Gq'FHIVZvKmlGWQ-j?Wj,RpR\nO-.-hSpIFGe&UaofHRzX:O h!FqmoDeMBQ-KyrxH.I;I$M\ntyxng,IAGJbThdAWzHul sAtlDo3TO\nF?wIgw\nfFDPPLI.imWNb;-JyKYxbbm&-A,iWmG'g;,J;prv-brHd?\n'l&kUzBQTUpZ\nt\n-Ku&ZfzhOD&tgylDXdfaKhyuvOcwVJ!i:aZw\nb;AqVg3l&IJ3zCXOzviwrQyCFgggx?W?y EWti.VUf:hXE\n?QkBKmypL'hVCz!QUPceqBHls$hVs,XQqSpC3WvWDqbPNPY3FrfDSLA\nDpezeeE?$qVTdsJ\nZXX.LhxXJ-\nJHza'C:DYpZ3rmclSLXxBII-CEzzvhi,yQrVjy;hD\ntEtZHFxK$Aq!hq&?KLv:boiz,VUkLglClvOyQsTPhZvk?DZ :yE&?GGSF!$!:h

### Ekspor Model Generator

In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [ ]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:GcVpNLAVMzxhK$IgKsRbbIsn; jiXnmj!o!EPboenr
?sOvoWeBNEst ?k.QWrsy,Si&N,:ON-n.Gz,Di:iV'myZkynLhckDLugs


# TUGAS

Gunakan tf.GradientTape untuk men track nilai gradient. Anda dapat mempelajari lebih lanjut tentang pendekatan ini dengan membaca eager execution guide.

Prosedurnya adalah :
1. Jalankan Model dan hitung loss dengan tf.GradientTape.
2. Hitung update dan terapkan pada model dengan optimizer

In [ ]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
    inputs, labels = inputs
    with tf.GradientTape() as tape:
      predictions = self(inputs, training=True)
      loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [ ]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [ ]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 16s 60ms/step - loss: 2.7178


In [ ]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
  start = time.time()

  mean.reset_states()
  for (batch_n, (inp, target)) in enumerate(dataset):
    logs = model.train_step([inp, target])
    mean.update_state(logs['loss'])

    if batch_n % 50 == 0:
      template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
      print(template)

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print()
  print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
  print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
  print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.2008
Epoch 1 Batch 50 Loss 2.0394
Epoch 1 Batch 100 Loss 1.9635
Epoch 1 Batch 150 Loss 1.9122

Epoch 1 Loss: 1.9977
Time taken for 1 epoch 18.32 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.8293
Epoch 2 Batch 50 Loss 1.7845
Epoch 2 Batch 100 Loss 1.7072
Epoch 2 Batch 150 Loss 1.6665

Epoch 2 Loss: 1.7191
Time taken for 1 epoch 11.97 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.6409
Epoch 3 Batch 50 Loss 1.5790
Epoch 3 Batch 100 Loss 1.5555
Epoch 3 Batch 150 Loss 1.5150

Epoch 3 Loss: 1.5562
Time taken for 1 epoch 12.18 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4786
Epoch 4 Batch 50 Loss 1.4734
Epoch 4 Batch 100 Loss 1.4461
Epoch 4 Batch 150 Loss 1.4716

Epoch 4 Loss: 1.4570
Time taken for 1 epoch 12.23 sec
_____________________________________________________________________

# Soal Tugas

Jelaskan kode diatas dan sebutkan perbedaanya dengan praktikum 2?

- Perbedaan antara kode tugas dan praktikum 2 terletak pada pendekatan pelatihan yang digunakan. Pada praktikum 2, digunakan pendekatan pelatihan sederhana menggunakan **'model.fit'**. Namun, dalam kode tugas, diterapkan pendekatan pelatihan yang lebih khusus dan kompleks. Dalam pendekatan khusus ini, didefinisikan metode **'train_step'** dalam turunan model, yang mengontrol pelatihan pada tingkat batch. Metode ini secara eksplisit menghitung nilai loss, gradien, dan dilakukan pembaruan bobot model. Selain itu, digunakan objek **'tf.metrics.Mean'** untuk menghitung rata-rata loss selama pelatihan. Jadi bisa disimpulkan kalau pendekatan ini memberi banyak kontrol dan fleksibilitas yang tinggi dalam pelatihan model